# Bronze Data Exploration

# Dimension 
- Business Partners - keys, partner_id
- Employees - keys, employee_id
- Dates - keys, datekey 
- Products -keys, product_id


# Fact
- Sales Order Iems - Detail, sale_order_items, sale_order_id
- Sales Order - Header - sale_order_id

In [0]:
%sql
SELECT * FROM db_bike.01_bronze.bronze_employees

In [0]:
%sql
SELECT * FROM db_bike.`01_bronze`.bronze_addresses

# Transformation: Employees & Business Partners

In [0]:
%sql
SELECT DISTINCT region FROM db_bike.`01_bronze`.bronze_addresses

In [0]:
%sql
SELECT 
  TRIM(b.EMPLOYEEID) AS employee_id
  ,TRIM(b.NAME_FIRST) AS first_name
  ,TRIM(b.NAME_MIDDLE) AS middle_name
  ,TRIM(b.NAME_LAST) AS last_name
  ,CASE
      TRIM(b.SEX)
      WHEN 'M' THEN 'Male'
      WHEN 'F' THEN 'Female'
      ELSE 'Unknown'
    END AS gender
  ,CONCAT(SUBSTRING(TRIM(b.NAME_FIRST), 1, 3),".",TRIM(b.NAME_LAST)) AS login_name
  ,CONCAT(SUBSTRING(TRIM(b.NAME_FIRST), 1, 3),".",TRIM(b.NAME_LAST),"@fontaine.com.ph") AS email_address
  ,TRIM(a.COUNTRY) AS country_name
  ,TRIM(a.REGION) AS region_name
  ,CASE 
      WHEN CAST(SUBSTRING(TRIM(b.VALIDITY_ENDDATE), 1, 4) AS INT) > YEAR(CURRENT_DATE()) THEN 1
      ELSE 0
  END AS is_active
FROM db_bike.01_bronze.bronze_employees b
INNER JOIN db_bike.01_bronze.bronze_addresses a ON b.ADDRESSID = a.ADDRESSID
WHERE
    CAST(SUBSTRING(TRIM(b.VALIDITY_ENDDATE), 1, 4) AS INT) > YEAR(CURRENT_DATE()); 

In [0]:
%sql
SELECT * FROM db_bike.01_bronze.bronze_business_partner

In [0]:
%sql
SELECT
  TRIM(b.PARTNERID) AS partner_id
  ,TRIM(b.COMPANYNAME) AS company_name
  ,TRIM(b.LEGALFORM) AS legal_form
  ,TRIM(a.COUNTRY) AS country_name
  ,TRIM(a.REGION) AS region_name
  ,CAST(b.PARTNERROLE AS INT) AS partner_role
  ,TRIM(b.EMAILADDRESS) AS email_address
  ,TRIM(b.PHONENUMBER) AS phone_number
  ,TRIM(b.FAXNUMBER) AS fax_number
  ,TRIM(b.WEBADDRESS) AS web_address
FROM db_bike.01_bronze.bronze_business_partner b
INNER JOIN db_bike.01_bronze.bronze_addresses a ON b.ADDRESSID = a.ADDRESSID

# Transformations for Products

In [0]:
%sql
SELECT * FROM db_bike.01_bronze.bronze_prod_cat_text

In [0]:
%sql
SELECT * FROM db_bike.`01_bronze`.bronze_prod_text

In [0]:
%sql
SELECT * FROM db_bike.01_bronze.bronze_prod_cat -- exclude

In [0]:
%sql
SELECT * FROM db_bike.`01_bronze`.bronze_prod

In [0]:
%sql
SELECT COUNT(*), PRODUCTID FROM db_bike.`01_bronze`.bronze_prod GROUP BY PRODUCTID
HAVING COUNT(*) > 1

In [0]:
%sql
SELECT 
  TRIM(a.PRODUCTID) AS product_id
  ,TRIM(a.TYPECODE) AS type_code
  ,TRIM(a.SUPPLIER_PARTNERID) AS partner_id
  ,CAST(TRIM(a.PRICE) AS DECIMAL(10,2)) AS list_price
  ,TRIM(a.CURRENCY) AS currency
  ,TRIM(a.PRODCATEGORYID) AS product_category_id
  ,TRIM(c.SHORT_DESCR) AS product_category
  ,CASE
     WHEN TRIM(b.MEDIUM_DESCR) IS NULL THEN TRIM(b.SHORT_DESCR)
     ELSE TRIM(b.MEDIUM_DESCR)
  END AS product_description
  ,CAST(TRIM(a.WEIGHTMEASURE) AS DECIMAL(10,2)) AS weight_measure
  ,TRIM(a.WEIGHTUNIT) AS weight_unit
FROM db_bike.01_bronze.bronze_prod a
INNER JOIN db_bike.01_bronze.bronze_prod_text b ON a.PRODUCTID = b.PRODUCTID AND b.LANGUAGE = 'EN'
INNER JOIN db_bike.01_bronze.bronze_prod_cat_text c ON c.PRODCATEGORYID = a.PRODCATEGORYID
WHERE TRIM(a.PRODUCTID) IS NOT NULL OR 
    CAST(TRIM(a.PRICE) AS DECIMAL(10,2)) IS NOT NULL
    OR CAST(TRIM(a.PRICE) AS DECIMAL(10,2)) <> 0


# Transformation Sales Tables

In [0]:
%sql
SELECT * FROM db_bike.01_bronze.bronze_sales_items

In [0]:
%sql
SELECT DISTINCT CURRENCY FROM db_bike.01_bronze.bronze_sales_items

In [0]:
%sql
SELECT 
  TRIM(SALESORDERID) AS sales_order_id
  ,TRIM(SALESORDERITEM) AS sales_order_item
  ,CAST(TRIM(DELIVERYDATE) AS INT) AS delivery_date
  ,TRIM(PRODUCTID) AS product_id
  ,CAST(TRIM(GROSSAMOUNT) AS DECIMAL(10,2)) AS gross_amount
  ,CAST(TRIM(NETAMOUNT) AS DECIMAL(10,2)) AS net_amount
  ,CAST(TRIM(TAXAMOUNT) AS DECIMAL(10,2)) AS tax_amount
  ,CAST(TRIM(QUANTITY) AS INT) AS quantity
FROM db_bike.01_bronze.bronze_sales_items

In [0]:
%sql
SELECT DISTINCT PARTNERID FROM db_bike.01_bronze.bronze_sales_order

In [0]:
%sql
SELECT * FROM db_bike.01_bronze.bronze_sales_order

In [0]:
%sql
SELECT DISTINCT BILLINGSTATUS FROM db_bike.01_bronze.bronze_sales_order

In [0]:
%sql
SELECT 
    TRIM(SALESORDERID) AS sales_order_id
    ,CAST(TRIM(CREATEDAT) AS INT) AS transaction_date
    ,TRIM(PARTNERID) AS partner_id
    ,CAST(TRIM(GROSSAMOUNT) AS DECIMAL(10, 2)) AS total_gross_amount
    ,CAST(TRIM(NETAMOUNT) AS DECIMAL(10, 2)) AS total_net_amount
    ,CAST(TRIM(TAXAMOUNT) AS DECIMAL(10, 2)) AS total_tax_amount
    ,CASE 
        TRIM(LIFECYCLESTATUS)
        WHEN 'C' THEN 'Completed Order'
        WHEN 'X' THEN 'Cancelled Order'
        WHEN 'I' THEN 'Incomplete Order'
        ELSE NULL
    END AS lifecycle_status
    ,CASE 
        TRIM(BILLINGSTATUS)
        WHEN 'C' THEN 'Billed'
        WHEN 'X' THEN 'Cancelled Bill'
        WHEN 'I' THEN 'Unbilled'
        ELSE NULL
    END AS billing_status
    ,CASE
        TRIM(DELIVERYSTATUS)
        WHEN 'C' THEN 'Order Delivered'
        WHEN 'X' THEN 'Cancelled Develvery'
        WHEN 'I' THEN 'Incomplete Delivery'
        ELSE NULL
    END AS delivery_status
    ,TRIM(CREATEDBY) AS created_by
    ,TRIM(BILLINGSTATUS)
    ,CASE 
        WHEN TRIM(BILLINGSTATUS) <> 'X' THEN 1
        ELSE 0
    END AS is_cancelled
FROM db_bike.01_bronze.bronze_sales_order